<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Disclaimer" data-toc-modified-id="Disclaimer-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Disclaimer</a></span></li><li><span><a href="#Intended-Outcomes" data-toc-modified-id="Intended-Outcomes-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Intended Outcomes</a></span></li><li><span><a href="#Preparation:-System-Setup" data-toc-modified-id="Preparation:-System-Setup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Preparation: System Setup</a></span><ul class="toc-item"><li><span><a href="#Install-the-IDE" data-toc-modified-id="Install-the-IDE-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Install the IDE</a></span></li><li><span><a href="#Install-the-opentrons-module" data-toc-modified-id="Install-the-opentrons-module-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Install the opentrons module</a></span></li><li><span><a href="#Keeping-the-opentrons-module-up-to-date" data-toc-modified-id="Keeping-the-opentrons-module-up-to-date-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Keeping the opentrons module up-to-date</a></span></li></ul></li><li><span><a href="#Setting-up-a-virtual-environment-for-simulating-a-robot" data-toc-modified-id="Setting-up-a-virtual-environment-for-simulating-a-robot-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Setting up a virtual environment for simulating a robot</a></span></li><li><span><a href="#Creating-labware-objects" data-toc-modified-id="Creating-labware-objects-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Creating labware objects</a></span></li><li><span><a href="#Additional-remarks" data-toc-modified-id="Additional-remarks-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Additional remarks</a></span></li></ul></div>

# Introduction
This workshop aims at making the OT-2 APIv2 as approachable as possible for people like me who have limited experience in programming. The workshop material below is in someway just a skimmed walkthrough of the [Opentrons OT-2 APIv2 docs](http://docs.opentrons.com/v2). However, this is not meant to be a OT-2 programming 101 course and so it does not follow the official order nor their recommendations. OT-2 users who wish to make the most out of the robot are encouraged to peruse the docs.

In the rest of the document, API refers to APIv2.

To summarize, our approach to the API is:
1. Build a virtual environment (a sandbox) to play around with features of the API  
\------ Draft starts ------
2. Unit test the labwares, wells, pipettes and commands and get instant readouts
3. Generate a human-trackable unit action, e.g. one single liquid transfer
4. (Loop that unit action to scale up the experiment)  
\------ Draft ends ------
5. Simulate the draft in the virtual environment, get the simulated output and check whether it is correct
6. Remove the virtual environment and convert the draft into a proper script
7. Do a final confirmation by simulating the script and exporting its formal runlog (__important for experiment documentation__)

To avoid creating a script from scratch every time, I will briefly share how I structure my scripts.

# Disclaimer
1. I have limited experience in programming and received little formal training. Coding practices may be unorthodox and even heretic in the eyes of properly trained programmers.
2. Methods of execution are in most cases suboptimal in performance. (see Additional Remarks #1)
3. Please forgive my ocassional loss of professionalism in language.


# Intended Outcomes

By the end of the workshop, we hope participants would be able to:

1. Read the documentation on APIv2, and know where to look for more information, and where to seek help if needed
2. Set up a virtual environment for drafting a Python protocol for OT-2 APIv2
3. Debug the code using the variable explorer and the console
4. Import the opentrons library to initialize a protocol object, create labware objects and pipettes within it and retrieve their information
5. Access wells of plates / racks
6. Instruct the pipettes to carry out liquid movement using pipette.transfer() and pipette.distribute(), and configure them with additional parameters
7. Create a custom pipetting step using building block commands
8. Use loops, lists and dictionaries together to scale up pipetting steps
9. Simulate the protocol in a PC and then converting it to a robot-executable one
10. Export simulate results to a readable-file.
11. Write the framework of practically useful protocols for automatable experiments 
12. Use Excel to create list of parsable commands to be copied and pasted into a protocol

# Preparation: System Setup

## Install the IDE
It is best to make use of an Integrated Development Environment (IDE) when writing codes. My personal experience suggests that  Spyder is most friendly to Python beginners and its Variable Explorer is tremendously helpful, both in debugging and in understanding how the API works. Some other workshops recommend Jupyter or PyCharm. Participants are welcomed to stick with the IDEs they are most comfortable with.

The relatively fool-proof way to get Spyder is to download and install [Anaconda](https://www.anaconda.com/distribution/). For advanced users, the accompanying site packages like pandas and NumPy comes in handy for interfacing experiment results and experiment automation. (This will not be covered here)

## Install the opentrons module

Once Anaconda is installed, participants should open Spyder, and install the opentrons module / library, by typing in the IPython console:  

`pip install opentrons`

To check if the opentrons is correctly installed, participants should try and see if they could get the module version from the  console.

In [18]:
import opentrons
print(opentrons.__version__)

3.16.1


Failure to do so should give an error message of `ModuleNotFoundError: No module named 'opentrons'`

## Keeping the opentrons module up-to-date
Opentrons update their API and library from time to time and it should happen together with OT-2 App and robot firmware upgrade. Always check if your opentrons module is the latest, and if needed, upgrade your opentrons module on your computer by typing in the terminal  

`pip install opentrons --upgrade`

# Setting up a virtual environment for simulating a robot

The following section corresponds to ["Simulating your scripts"](https://docs.opentrons.com/v2/writing.html#simulating-your-scripts) from the API docs.

Let's start by typing the following into the Spyder Editor.

In [1]:
from opentrons import simulate
protocol = simulate.get_protocol_api(version = '2.2')

Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


The `protocol` object is the starting point of all robot related actions, like choosing the pipette and putting labwares on deck. It also stores the simulated output of the robot. To inspect its functions and attributes, run `dir(protocol)` in the IPython console.

# Creating labware objects

The following section corresponds to ["Labware"](https://docs.opentrons.com/v2/new_labware.html#labware) and ["Default labware"](https://docs.opentrons.com/v2/new_labware.html#default-labware) from the Opentrons docs.

Opentrons' definition of labware includes both the mult-well plates/racks/reserviors and the tip racks. They are loaded onto the robot via the function:
`var = protocol.load_labware(load_name, location)`  
- load_name: what is your labware?  
For example
- slot: which of the slot (1-11) are you placing it?


In [11]:
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)

In [13]:
protocol = simulate.get_protocol_api(version = '2.2')

C:\Users\User\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\User\.opentrons\robot_settings.json not found. Loading defaults


In [12]:
plate

Corning 96 Well Plate 360 µL Flat on 1

# Additional remarks

1. It is faster to use `pipette.transfer()` with list of well objects than looping a a single `pipette.transfer()` function multiple times. The difference in speed in unnoticeable on a PC but could mean 4-5 minutes of wait when API of a long protocol is compiled on a Pi-powered robot.

The following example illustrates a comparison of the two methods, both trying to pipette 200 µL from well A1 to A7, B1 to B7, ... H6 to H12, for a total of 48 transfer steps.

A single complex function processes multi-to-multi transfers much more efficiently, at the expense of losing fine-tuned control at the atomic movement level.

In [1]:
# Set up
import time
from opentrons import simulate
protocol = simulate.get_protocol_api('2.2')
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', location = '1')
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', location = '2')
pipette = protocol.load_instrument('p300_single', mount = 'right', tip_racks = [tip_rack])
print('Run time comparison:')

# Method 1: List for looping pipette.transfer
transfer_info = {x: x + 48 for x in range(0,48)}

start_time = time.time()
for source_index, dest_index in transfer_info.items():
    pipette.transfer(200, plate.wells(source_index), plate.wells(dest_index))
end_time = time.time()
time_diff = end_time - start_time
print('Loop method: ' + str(time_diff))

# Method 2: Looping through a series of wells
source_wells = plate.columns()[0:3]
dest_wells = plate.columns()[4:7]
start_time = time.time()
pipette.transfer(200, source_wells, dest_wells)
end_time = time.time()
time_diff = end_time - start_time
print('Well object method: ' + str(time_diff))

Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


C:\Users\s1635543\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\s1635543\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\s1635543\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\s1635543\.opentrons\robot_settings.json not found. Loading defaults


Run time comparison:
Loop method: 0.6629588603973389
Well object method: 0.11969971656799316


2. To interface with external files like .csv, the API provides a method but that is not recommended.
That is however not recommended.